In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [2]:
import pandas as pd

In [41]:
df = pd.read_csv("../data.csv", sep=";")

df.tail()

,Product,Review
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld,I loved this product. But they only seem to la...
5,L'Or Espresso Café,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


## LLMChain

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [21]:
llm = ChatOpenAI(temperature=0.9)

In [22]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

In [23]:
chain = LLMChain(llm=llm, prompt=prompt)

In [24]:
product = "Queen Size Sheet Set"
chain.run(product)

'"RoyalRest Bedding" or "RegalDreams Linens"'

## SimpleSequentialChain

In [25]:
from langchain.chains import SimpleSequentialChain

In [30]:
llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt)

chain_one

LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=ChatPromptTemplate(input_variables=['product'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['product'], output_parser=None, partial_variables={}, template='What is the best name to describe     a company that makes {product}?', template_format='f-string', validate_template=True), additional_kwargs={})]), llm=ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.9, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, openai_proxy=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None), output_key='text')

In [29]:
second_prompt = ChatPromptTemplate.from_template(
    "Write 20 words description for the following company: {company_name}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [31]:
seq_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

seq_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Royal Comfort Linens"
Royal Comfort Linens is a premium bedding company, providing luxurious and high-quality linens for ultimate comfort and relaxation.

> Finished chain.


'Royal Comfort Linens is a premium bedding company, providing luxurious and high-quality linens for ultimate comfort and relaxation.'

## SequentialChain

In [32]:
from langchain.chains import SequentialChain

In [33]:
llm = ChatOpenAI(temperature=0.9)

In [34]:
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:" "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

In [35]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:" "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [36]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

In [37]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")

In [44]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True,
)

In [45]:
overall_chain(df.Review[5])



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...Vieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones from the store and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the taste and foam quality of the product, suspecting that it may be either an old batch or a counterfeit item compared to the ones purchased from the store.',
 'followup_message': "Suite à votre commentaire, nous sommes désolés d'apprendre que vous n'êtes pas satisfait du goût et de la qualité de la mousse de notre produit. Nous comprenons votre préoccupation quant à la possibilité qu'il s'agisse d'un lot périmé ou d'un article contrefait par rapport à ceux achetés en magasin.\n\nNous tenons à vous assurer que nous prenons ces problèmes très au sérieux et que nous nous engageons à fournir à 

## Router Chain

In [48]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [50]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
    {
        "name": "history",
        "description": "Good for answering history questions",
        "prompt_template": history_template,
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template,
    },
]

In [52]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [53]:
llm = ChatOpenAI(temperature=0)

In [55]:
destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [57]:
print(destinations_str)

physics: Good for answering questions about physics
math: Good for answering math questions
history: Good for answering history questions
computer science: Good for answering computer science questions


In [58]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [59]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [61]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str,
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [62]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [63]:
chain.run("What is the weight of Sun?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is the weight of the Sun?'}
> Finished chain.


'The weight of an object is the force exerted on it due to gravity. In the case of the Sun, it is not appropriate to talk about its weight because weight is a measure of the force of gravity on an object by another object. Instead, we can talk about the mass of the Sun, which is approximately 1.989 × 10^30 kilograms.'

In [64]:
chain.run("What is 1+1?")



> Entering new MultiPromptChain chain...
math: {'input': 'What is 1+1?'}
> Finished chain.


"Thank you for your kind words! The question you've asked is a simple addition problem. To find the sum of 1+1, we add the two numbers together. \n\n1 + 1 = 2\n\nTherefore, the answer to 1+1 is 2."

In [67]:
chain.run("When America was discovered?")



> Entering new MultiPromptChain chain...
history: {'input': 'When was America discovered?'}
> Finished chain.


'The discovery of America is a complex and debated topic among historians. While it is commonly believed that Christopher Columbus discovered America in 1492, it is important to note that indigenous peoples had already been living in the Americas for thousands of years before Columbus arrived. Additionally, Norse explorers, led by Leif Erikson, are believed to have reached North America around the 11th century, establishing temporary settlements in areas such as Newfoundland, Canada.\n\nFurthermore, it is crucial to acknowledge that the concept of "discovery" itself is Eurocentric and disregards the existence and knowledge of indigenous civilizations. The Americas were already inhabited by diverse indigenous cultures with rich histories and civilizations, such as the Aztecs, Mayans, and Incas, long before European contact.\n\nTherefore, it is more accurate to say that America was encountered or encountered by Europeans in 1492, rather than discovered.'

In [68]:
chain.run("Was Australia being inhabited before it was discovered?")



> Entering new MultiPromptChain chain...
history: {'input': 'Was Australia inhabited before it was discovered?'}
> Finished chain.


'Thank you for your kind words! As a historian, I can provide you with an answer to your question.\n\nYes, Australia was indeed inhabited by Indigenous peoples long before it was "discovered" by Europeans. The Aboriginal and Torres Strait Islander peoples have a rich and diverse history that stretches back at least 65,000 years. They had developed complex societies, cultures, and languages, and had established deep connections with the land.\n\nThe arrival of Europeans in Australia, beginning with the Dutch in the early 17th century, marked the beginning of a new chapter in Australian history. However, it is important to note that the continent was not an empty land waiting to be discovered. The Indigenous peoples had already established their own systems of governance, trade, and cultural practices.\n\nEuropean colonization had a profound impact on the Indigenous peoples of Australia. It led to dispossession of land, violence, disease, and the erosion of traditional cultures. The hist